In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import time
import os
import yaml

In [125]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir"

# Function to load yaml configuration file
def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")


image = "/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp"
img = cv2.imread(image)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray,45,255,cv2.THRESH_BINARY)[1]

t_lower = 100  # Lower Threshold
t_upper = 200  # Upper threshold
aperture_size = 5  # Aperture size
L2Gradient = True
edge = cv2.Canny(thresh, t_lower, t_upper,apertureSize = aperture_size, L2gradient = L2Gradient)
y, x = edge.shape

n = 8
start_x = 2
start_y = 7
edge1 = edge[start_y * y // n: (start_y + 1) * y // n, start_x * x // n:(start_x + 1) * x // n]
output = cv2.connectedComponentsWithStats(edge1, 8, cv2.CV_32S)
(numLabels, labels, stats, centroids) = output


# loop over the number of unique connected component labels
for i in range(0, numLabels):
    if i == 0:
        text = "examining component {}/{} (background)".format(
            i + 1, numLabels)
    # otherwise, we are examining an actual connected component
    else:
        text = "examining component {}/{}".format( i + 1, numLabels)
    x_ = stats[i, cv2.CC_STAT_LEFT]
    y_ = stats[i, cv2.CC_STAT_TOP]
    w_ = stats[i, cv2.CC_STAT_WIDTH]
    h_ = stats[i, cv2.CC_STAT_HEIGHT]
    area = stats[i, cv2.CC_STAT_AREA]
    (cX, cY) = centroids[i]

    keepWidth = w_ > 2
    keepHeight = h_ > 2
    keepArea = area > 50

    if all((keepWidth, keepHeight, keepArea)):
        componentMask = (labels == i).astype("uint8") * 255
        cv2.imwrite(f"/home/office5/Image_Code/Ketan_sir/try1_connected_component/{i}.png",componentMask)

In [45]:

origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
thresh = cv2.threshold(origin_img,45,255,cv2.THRESH_BINARY)[1]

# # Setting All parameters
t_lower = 100  # Lower Threshold
t_upper = 200  # Upper threshold
aperture_size = 7  # Aperture size
L2Gradient = True
    
# # Applying the Canny Edge filter
# # with Custom Aperture Size
edge = cv2.Canny(thresh, t_lower, t_upper, L2gradient = L2Gradient)
y, x = edge.shape

n = 8
start_x = 2
start_y = 7
edge1 = origin_img[2744:3136,1824:2736]
print(edge1.shape)
# show(edge1)

l = []
x = []
y = []
for img1 in sorted(glob.glob('/home/office5/Image_Code/Ketan_sir/try_connected_component/*.png'), key=len):
    if img1 != '/home/office5/Image_Code/Ketan_sir/try_connected_component/0.png':
        img_name = img1.split('/')[-1]
        img = cv2.imread(img1)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        points = []
        # for i in range(len(coords)):
        #     if(y_min == coords[i][0]):
        #         pass
        #     else:
        #         points.append([coords[i-1][0], coords[i-1][1]]) 
        #         break

        # points.append([coords[0][0] , coords[0][1]])
        
        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)
        corner_point = points.copy()
        points[0] = points[0] + np.array([2744,1824]) 
        points[1] = points[1] + np.array([2744,1824]) 

        for i in range(3):
            try:
                upp_down_a = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]+1][points[i][1]]))
                left_right_b = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]][points[i][1]+1]))
                
                up_right = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]][points[i][1]+1]))
                left_down = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]+1][points[i][1]]))

                if(upp_down_a>=30 and left_right_b>=20 and up_right>15 and left_down>15):
                    # print(points[i])
                    print(img_name,upp_down_a,left_right_b,up_right,left_down,corner_point[i])
                    # print(edge1[106,42])
                    edge1[corner_point[i][0],corner_point[i][1]] = 255
                    origin_img[points[i][0],points[i][1]] = 255

            except:
                pass
cv2.imwrite('extreme_point1.png',edge1)
cv2.imwrite('extreme_point_original1.png',origin_img)



(392, 912)
68.png [[35]] [[36]] [[28]] [[43]] [112 454]
37.png [[44]] [[21]] [[32]] [[33]] [ 51 526]
37.png [[44]] [[21]] [[32]] [[33]] [ 51 526]
45.png [[56]] [[24]] [[36]] [[44]] [ 31 459]
69.png [[30]] [[42]] [[42]] [[30]] [33 28]
71.png [[30]] [[42]] [[42]] [[30]] [33 28]
64.png [[39]] [[48]] [[43]] [[44]] [113 453]
32.png [[60]] [[21]] [[35]] [[46]] [  9 534]
60.png [[55]] [[32]] [[46]] [[41]] [ 45 417]
59.png [[56]] [[24]] [[43]] [[37]] [ 46 415]
190.png [[56]] [[23]] [[48]] [[31]] [290 763]
447.png [[37]] [[21]] [[39]] [[19]] [344 549]
455.png [[37]] [[21]] [[39]] [[19]] [344 549]
187.png [[42]] [[50]] [[52]] [[40]] [233 738]
297.png [[37]] [[34]] [[19]] [[52]] [262 728]
297.png [[56]] [[23]] [[17]] [[16]] [291 747]
273.png [[35]] [[46]] [[44]] [[37]] [286 413]
511.png [[50]] [[41]] [[45]] [[46]] [375 534]
307.png [[55]] [[27]] [[42]] [[40]] [292 361]
244.png [[35]] [[46]] [[44]] [[37]] [286 413]
458.png [[49]] [[23]] [[34]] [[38]] [376 465]
285.png [[52]] [[33]] [[35]] [[50]] [

True

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import time
import os
import yaml

In [30]:

origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
thresh = cv2.threshold(origin_img,45,255,cv2.THRESH_BINARY)[1]



CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir"

def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

# # Setting All parameters
t_lower = 100  # Lower Threshold
t_upper = 200  # Upper threshold
aperture_size = 7  # Aperture size
L2Gradient = True
    
# # Applying the Canny Edge filter
# # with Custom Aperture Size
edge = cv2.Canny(thresh, t_lower, t_upper, L2gradient = L2Gradient)
y, x = edge.shape

n = 8
start_x = 2
start_y = 7
edge1 = origin_img[2744:3136,1824:2736]
print(edge1.shape)
# show(edge1)

l = []
x = []
y = []
right_angle_point = []

for img1 in sorted(glob.glob('/home/office5/Image_Code/Ketan_sir/try1_connected_component/*.png'), key=len):
    if img1 != '/home/office5/Image_Code/Ketan_sir/try1_connected_component/0.png':
        img_name = img1.split('/')[-1]
        img = cv2.imread(img1)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        # gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = np.float32(gray)
        dst = cv2.cornerHarris(gray,2,3,0.04)
        ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
        dst = np.uint8(dst)
        # plt.imshow(dst)
        # plt.show()

        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

        for i in range(len(corners)):
            for j in range(1,config['measure_pixel']):
                hei_inc = int(np.int0(corners[i])[1]) + j
                wid_inc = int(np.int0(corners[i])[0]) + j
                hei_dec = int(np.int0(corners[i])[1]) - j
                wid_dec = int(np.int0(corners[i])[0]) - j
                same_hei = int(np.int0(corners[i])[1])
                same_wid = int(np.int0(corners[i])[0])

                check_extreme_point = []
                try:
                    if(gray[same_hei][same_wid]!=255):
                        break
                    else:
                        if( (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_dec]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_dec]!=255)):
                            break
                        else:
                            check_extreme_point.append(1)        
                except:
                    print("out of image")

            if(len(check_extreme_point)==1):

                # print("#-------------------------New Points -------------------------------#")

                edge1[same_hei,same_wid] = 255
                right_angle_point.append([same_hei,same_wid])

        # print(right_angle_point)

        points = []
        
        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)
        corner_point = points.copy()
        points[0] = points[0] + np.array([2744,1824]) 
        points[1] = points[1] + np.array([2744,1824]) 

        for i in range(3):
            try:
                upp_down_a = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]+1][points[i][1]]))
                left_right_b = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]][points[i][1]+1]))
                
                up_right = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]][points[i][1]+1]))
                left_down = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]+1][points[i][1]]))
                
                if(upp_down_a>=30 and left_right_b>=20 and up_right>15 and left_down>15):
                    # print(points[i])
                    print(img_name,upp_down_a,left_right_b,up_right,left_down,corner_point[i])
                    # print(edge1[106,42])
                    edge1[corner_point[i][0],corner_point[i][1]] = 255
                    origin_img[points[i][0],points[i][1]] = 255
                    # print(edge1[106,42])
                    x.append(points[i][0])
                    y.append(points[i][1])

            except:
                pass

print(right_angle_point)
cv2.imwrite('extreme_point2.png',edge1)
cv2.imwrite('extreme_point_original2.png',origin_img)



(392, 912)
37.png [[44]] [[21]] [[32]] [[33]] [ 51 526]
37.png [[44]] [[21]] [[32]] [[33]] [ 51 526]
64.png [[39]] [[48]] [[43]] [[44]] [113 453]
98.png [[48]] [[25]] [[8]] [[15]] [ 88 248]
58.png [[40]] [[24]] [[4]] [[12]] [123  21]
77.png [[43]] [[38]] [[2]] [[7]] [ 56 395]
59.png [[56]] [[24]] [[43]] [[37]] [ 46 415]
190.png [[56]] [[23]] [[48]] [[31]] [290 763]
454.png [[65]] [[38]] [[14]] [[13]] [377 496]
297.png [[37]] [[34]] [[19]] [[52]] [262 728]
297.png [[56]] [[23]] [[17]] [[16]] [291 747]
273.png [[35]] [[46]] [[44]] [[37]] [286 413]
307.png [[55]] [[27]] [[42]] [[40]] [292 361]
277.png [[52]] [[33]] [[35]] [[50]] [262 612]
277.png [[53]] [[36]] [[50]] [[39]] [265 671]
317.png [[48]] [[22]] [[44]] [[26]] [294 252]
301.png [[40]] [[79]] [[22]] [[17]] [286 191]
301.png [[48]] [[29]] [[43]] [[34]] [294 215]
413.png [[62]] [[38]] [[13]] [[11]] [378 388]
256.png [[43]] [[80]] [[17]] [[20]] [223 399]
256.png [[36]] [[69]] [[54]] [[51]] [243 408]
218.png [[41]] [[23]] [[5]] [[13]]

True

In [129]:

origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
thresh = cv2.threshold(origin_img,45,255,cv2.THRESH_BINARY)[1]



CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir"

def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

# # Setting All parameters
t_lower = 100  # Lower Threshold
t_upper = 200  # Upper threshold
aperture_size = 7  # Aperture size
L2Gradient = True
    
# # Applying the Canny Edge filter
# # with Custom Aperture Size
edge = cv2.Canny(thresh, t_lower, t_upper, L2gradient = L2Gradient)
y, x = edge.shape

n = 8
start_x = 2
start_y = 7
edge1 = origin_img[2744:3136,1824:2736]
print(edge1.shape)
# show(edge1)
start_x1 = start_x * x // n
end_x1 = (start_x + 1) * x // n
start_y1 = start_y * y // n
end_y1 = (start_y + 1) * y // n
height,width = end_y1 - start_y1,end_x1 - start_x1
# print(start_x1,start_y1)

l = []
x = []
y = []
right_angle_point = []
first_block, second_block, third_block, fourth_block = [],[],[],[]
all_point = []


for img1 in sorted(glob.glob('/home/office5/Image_Code/Ketan_sir/try1_connected_component/*.png'), key=len):
    if img1 != '/home/office5/Image_Code/Ketan_sir/try1_connected_component/0.png':
        img_name = img1.split('/')[-1]
        img = cv2.imread(img1)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        # gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = np.float32(gray)
        dst = cv2.cornerHarris(gray,2,3,0.04)
        ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
        dst = np.uint8(dst)
        # plt.imshow(dst)
        # plt.show()

        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

        for i in range(len(corners)):
            for j in range(1,config['measure_pixel']):
                hei_inc = int(np.int0(corners[i])[1]) + j
                wid_inc = int(np.int0(corners[i])[0]) + j
                hei_dec = int(np.int0(corners[i])[1]) - j
                wid_dec = int(np.int0(corners[i])[0]) - j
                same_hei = int(np.int0(corners[i])[1])
                same_wid = int(np.int0(corners[i])[0])

                check_extreme_point = []
                try:
                    if(gray[same_hei][same_wid]!=255):
                        break
                    else:
                        if( (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_dec]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_dec]!=255)):
                            break
                        else:
                            check_extreme_point.append(1)        
                except:
                    print("out of image")

            if(len(check_extreme_point)==1):

                lw,th,rw,bh=1,1,1,1
                countlw,countth,countrw,countbh=0,0,0,0

                # For left width count store value
                wid_dec = same_wid- lw
                while(gray[same_hei][wid_dec]==255):
                    wid_dec = same_wid - lw
                    countlw+=1
                    lw+=1

                # For top height count store value
                hei_dec = same_hei- th
                while(gray[hei_dec][same_wid]==255):
                    hei_dec = same_hei - th
                    countth+=1
                    th+=1

                # For right width count store value
                wid_inc = same_wid + rw
                while(gray[same_hei][wid_inc]==255):
                    wid_inc = same_wid + rw
                    countrw+=1
                    rw+=1

                # For bottom height count store value
                hei_inc = same_hei + bh
                while(gray[hei_inc][same_wid]==255):
                    hei_inc = same_hei + bh
                    countbh+=1
                    bh+=1
                
                all_point.append([same_hei,same_wid])

                if(same_hei< height//2 and same_wid < width//2):
                    first_block.append([same_hei,same_wid])
                elif(same_hei< height//2 and same_wid > width//2):  
                    second_block.append([same_hei,same_wid])  
                elif(same_hei > height//2 and same_wid < width//2):
                    third_block.append([same_hei,same_wid]) 
                elif(same_hei > height//2 and same_wid > width//2):
                    fourth_block.append([same_hei,same_wid])
                else:
                    print("No element")
                    

        # print(right_angle_point)

        points = []
        
        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)
        corner_point = points.copy()
        points[0] = points[0] + np.array([2744,1824]) 
        points[1] = points[1] + np.array([2744,1824]) 

        for i in range(3):
            try:
                upp_down_a = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]+1][points[i][1]]))
                left_right_b = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]][points[i][1]+1]))
                
                up_right = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]][points[i][1]+1]))
                left_down = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]+1][points[i][1]]))
                
                if(upp_down_a>=30 and left_right_b>=20):
                    edge1[corner_point[i][0],corner_point[i][1]] = 255
                    origin_img[points[i][0],points[i][1]] = 255

                    all_point.append([corner_point[i][0],corner_point[i][1]])

                    if(corner_point[i][0]< height//2 and corner_point[i][1] < width//2):
                        first_block.append([corner_point[i][0],corner_point[i][1]])
                    elif(corner_point[i][0]< height//2 and corner_point[i][1] > width//2):  
                        second_block.append([corner_point[i][0],corner_point[i][1]])  
                    elif(corner_point[i][0] > height//2 and corner_point[i][1] < width//2):
                        third_block.append([corner_point[i][0],corner_point[i][1]]) 
                    elif(corner_point[i][0] > height//2 and corner_point[i][1] > width//2):
                        fourth_block.append([corner_point[i][0],corner_point[i][1]])
                    else:
                        print("No element")
                    
            except:
                pass

# print(right_angle_point)
# cv2.imwrite('extreme_point2.png',edge1)
# cv2.imwrite('extreme_point_original2.png',origin_img)

print("First block",first_block)
print("Second Block",second_block)
print("Third Block",third_block)
print("Fourth Block",fourth_block)

(392, 912)
out of image
out of image
out of image
out of image
First block [[46, 415], [46, 415], [113, 453], [113, 453], [132, 208], [115, 211], [115, 211], [160, 176]]
Second Block [[51, 527]]
Third Block [[228, 321], [228, 321], [228, 321], [287, 320], [287, 298], [214, 250], [230, 213], [230, 213], [230, 213], [260, 213], [260, 213], [260, 213], [287, 212], [286, 380], [286, 380], [287, 320], [269, 228], [269, 228], [332, 436], [287, 212], [237, 218], [240, 240], [287, 298], [371, 400]]
Fourth Block [[226, 545], [226, 545], [226, 545], [285, 544], [211, 743], [211, 743], [211, 743], [238, 742], [254, 720], [254, 720], [254, 720], [283, 719], [283, 761], [238, 742], [283, 719], [283, 761], [285, 495], [285, 495], [370, 463], [370, 509], [370, 509], [274, 586], [232, 667], [246, 695], [370, 463], [370, 509], [370, 509], [262, 612]]


In [130]:
a = np.array(sorted(all_point))
s = []
for i in range(len(a)-1):
    m = cv2.absdiff(a[i+1],a[i])
    if(m[0]>2 or m[1]>2):
        s.append(a[i])
s[0]

array([ 46, 415])

In [131]:
# For showing difeerent color on extreme point
origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
edge1 = origin_img[2744:3136,1824:2736]

for i in range(len(s)):
    edge1[s[i][0],s[i][1]] = 255


s = s + np.array([2744,1824])
cv2.imwrite('one_point.png',edge1)

True

In [124]:
Anna2_img = "/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp"
Anna2_img = cv2.imread(Anna2_img,0)

Anna3_img = "/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp"
Anna3_img = cv2.imread(Anna3_img,0)


second_img_point = []
for i in range(len(s)):
    ori_img_loc_x = s[i][1]
    ori_img_loc_y = s[i][0]
    x1_f = ori_img_loc_x - 5
    x2_f = ori_img_loc_x + 5
    y1_f = ori_img_loc_y - 5
    y2_f = ori_img_loc_y + 5

    img_crop1  = Anna2_img[y1_f:y2_f,x1_f:x2_f]
    img_first = img_crop1.astype(np.int8)
    print("Original",ori_img_loc_y,ori_img_loc_x)

    x1_s = ori_img_loc_x - 10
    x2_s = ori_img_loc_x + 10
    y1_s = ori_img_loc_y - 10
    y2_s = ori_img_loc_y + 10

    img_crop2  = Anna3_img[y1_s:y2_s,x1_s:x2_s]
    img_snd = img_crop2.astype(np.int8)

    dict1 = {}
    for i in range(10):
        for j in range(10):
            dict1[i,j] = np.mean(np.abs(img_first  - img_snd[i:i+10,j:j+10]))
    # print(dict1)
    i,j = min(dict1, key=dict1.get)

    print(j,i)
    s_img_extr_point_x = x1_s + 10 - 5 + j # 10 for each block of first image and -5 for adding from width and height of first image
    s_img_extr_point_y =y1_s + 10 - 5 + i

    second_img_point.append([s_img_extr_point_y,s_img_extr_point_x])
    print("From second image",s_img_extr_point_y,s_img_extr_point_x)

Original 2790 2239
5 5
From second image 2790 2239
Original 2795 2351
5 5
From second image 2795 2351
Original 2857 2277
5 5
From second image 2857 2277
Original 2955 2567
5 5
From second image 2955 2567
Original 2970 2369
5 5
From second image 2970 2369
Original 2972 2145
5 5
From second image 2972 2145
Original 2974 2037
5 5
From second image 2974 2037
Original 2976 2491
5 5
From second image 2976 2491
Original 2981 2042
5 5
From second image 2981 2042
Original 2982 2566
5 5
From second image 2982 2566
Original 2984 2064
5 5
From second image 2984 2064
Original 2998 2544
5 5
From second image 2998 2544
Original 3004 2037
5 5
From second image 3004 2037
Original 3027 2543
5 5
From second image 3027 2543
Original 3027 2585
5 5
From second image 3027 2585
Original 3029 2319
5 5
From second image 3029 2319
Original 3029 2368
5 5
From second image 3029 2368
Original 3030 2204
5 5
From second image 3030 2204
Original 3031 2036
5 5
From second image 3031 2036
Original 3031 2122
5 5
From sec

In [85]:
second_img_point[0]

[2752, 2360]

In [86]:
origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna3.bmp')
edge2 = origin_img[2744:3136,1824:2736]

second_img_point = second_img_point - np.array([2744,1824])

for i in range(len(second_img_point)):
    edge2[second_img_point[i][0],second_img_point[i][1]] = [0,0,255]

cv2.imwrite('snd_img_extr.png',edge2)

True

# redpoint and green circle


In [6]:

origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
thresh = cv2.threshold(origin_img,45,255,cv2.THRESH_BINARY)[1]



CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir"

def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

# # Setting All parameters
t_lower = 100  # Lower Threshold
t_upper = 200  # Upper threshold
aperture_size = 7  # Aperture size
L2Gradient = True
    
# # Applying the Canny Edge filter
# # with Custom Aperture Size
edge = cv2.Canny(thresh, t_lower, t_upper, L2gradient = L2Gradient)
y, x = edge.shape

n = 8
start_x = 2
start_y = 7
edge1 = origin_img[2744:3136,1824:2736]
print(edge1.shape)
# show(edge1)
start_x1 = start_x * x // n
end_x1 = (start_x + 1) * x // n
start_y1 = start_y * y // n
end_y1 = (start_y + 1) * y // n
height,width = end_y1 - start_y1,end_x1 - start_x1
# print(start_x1,start_y1)

l = []
x = []
y = []
right_angle_point = []
first_block, second_block, third_block, fourth_block = [],[],[],[]
all_point = []


for img1 in sorted(glob.glob('/home/office5/Image_Code/Ketan_sir/try_connected_component/*.png'), key=len):
    if img1 != '/home/office5/Image_Code/Ketan_sir/try_connected_component/0.png':
        img_name = img1.split('/')[-1]
        img = cv2.imread(img1)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        # gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = np.float32(gray)
        dst = cv2.cornerHarris(gray,2,3,0.04)
        ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
        dst = np.uint8(dst)
        # plt.imshow(dst)
        # plt.show()

        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

        for i in range(len(corners)):
            for j in range(1,config['measure_pixel']):
                hei_inc = int(np.int0(corners[i])[1]) + j
                wid_inc = int(np.int0(corners[i])[0]) + j
                hei_dec = int(np.int0(corners[i])[1]) - j
                wid_dec = int(np.int0(corners[i])[0]) - j
                same_hei = int(np.int0(corners[i])[1])
                same_wid = int(np.int0(corners[i])[0])

                check_extreme_point = []
                try:
                    if(gray[same_hei][same_wid]!=255):
                        break
                    else:
                        if( (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_dec]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_dec]!=255)):
                            break
                        else:
                            check_extreme_point.append(1)        
                except:
                    print("out of image")

            if(len(check_extreme_point)==1):

                lw,th,rw,bh=1,1,1,1
                countlw,countth,countrw,countbh=0,0,0,0

                # For left width count store value
                wid_dec = same_wid- lw
                while(gray[same_hei][wid_dec]==255):
                    wid_dec = same_wid - lw
                    countlw+=1
                    lw+=1

                # For top height count store value
                hei_dec = same_hei- th
                while(gray[hei_dec][same_wid]==255):
                    hei_dec = same_hei - th
                    countth+=1
                    th+=1

                # For right width count store value
                wid_inc = same_wid + rw
                while(gray[same_hei][wid_inc]==255):
                    wid_inc = same_wid + rw
                    countrw+=1
                    rw+=1

                # For bottom height count store value
                hei_inc = same_hei + bh
                while(gray[hei_inc][same_wid]==255):
                    hei_inc = same_hei + bh
                    countbh+=1
                    bh+=1
                
                all_point.append([same_hei,same_wid])

                if(same_hei< height//2 and same_wid < width//2):
                    first_block.append([same_hei,same_wid])
                elif(same_hei< height//2 and same_wid > width//2):  
                    second_block.append([same_hei,same_wid])  
                elif(same_hei > height//2 and same_wid < width//2):
                    third_block.append([same_hei,same_wid]) 
                elif(same_hei > height//2 and same_wid > width//2):
                    fourth_block.append([same_hei,same_wid])
                else:
                    print("No element")
                    

        # print(right_angle_point)

        points = []
        
        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)
        corner_point = points.copy()

        if(len(points)==0):
            break
        elif(len(points)==1):
            points[0] = points[0] + np.array([2744,1824]) 
        elif(len(points)==2):
            points[0] = points[0] + np.array([2744,1824]) 
            points[1] = points[1] + np.array([2744,1824])
        elif(len(points)==3):
            points[0] = points[0] + np.array([2744,1824]) 
            points[1] = points[1] + np.array([2744,1824])
            points[2] = points[2] + np.array([2744,1824])
        elif(len(points)==4):
            points[0] = points[0] + np.array([2744,1824]) 
            points[1] = points[1] + np.array([2744,1824])
            points[2] = points[2] + np.array([2744,1824])
            points[3] = points[3] + np.array([2744,1824])
        else:
            print("point not found or max over")
            pass



        for i in range(len(points)):
            try:
                upp_down_a = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]+1][points[i][1]]))
                left_right_b = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]][points[i][1]+1]))
                
                up_right = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]][points[i][1]+1]))
                left_down = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]+1][points[i][1]]))
                
                if(upp_down_a>=30 and left_right_b>=20 and up_right>20 and left_down>20):
                    edge1[corner_point[i][0],corner_point[i][1]] = 255
                    origin_img[points[i][0],points[i][1]] = 255

                    all_point.append([corner_point[i][0],corner_point[i][1]])

                    if(corner_point[i][0]< height//2 and corner_point[i][1] < width//2):
                        first_block.append([corner_point[i][0],corner_point[i][1]])
                    elif(corner_point[i][0]< height//2 and corner_point[i][1] > width//2):  
                        second_block.append([corner_point[i][0],corner_point[i][1]])  
                    elif(corner_point[i][0] > height//2 and corner_point[i][1] < width//2):
                        third_block.append([corner_point[i][0],corner_point[i][1]]) 
                    elif(corner_point[i][0] > height//2 and corner_point[i][1] > width//2):
                        fourth_block.append([corner_point[i][0],corner_point[i][1]])
                    else:
                        print("No element")
                    
            except:
                pass

# print(right_angle_point)
cv2.imwrite('extreme_point2.png',edge1)
cv2.imwrite('extreme_point_original2.png',origin_img)

print("First block",first_block)
print("Second Block",second_block)
print("Third Block",third_block)
print("Fourth Block",fourth_block)

(392, 912)
First block [[112, 454], [33, 28], [33, 28], [113, 453], [45, 417], [46, 415]]
Second Block [[51, 526], [51, 526], [31, 459], [9, 534]]
Third Block [[286, 413], [292, 361], [286, 381], [286, 413], [292, 453], [209, 301], [294, 252], [294, 215], [255, 353], [269, 358], [286, 380], [243, 408], [332, 436], [287, 212], [371, 400], [234, 409], [376, 442], [294, 453], [243, 408], [287, 298], [209, 199], [307, 436], [287, 320], [293, 383], [237, 218], [240, 240]]
Fourth Block [[290, 763], [344, 549], [233, 738], [233, 738], [285, 495], [262, 735], [262, 735], [375, 534], [376, 465], [262, 612], [265, 671], [262, 612], [265, 671], [287, 688], [292, 764], [291, 548], [285, 544], [308, 533], [375, 574], [232, 667], [370, 531], [369, 509], [205, 596], [370, 509], [370, 509], [376, 512], [375, 573], [231, 735], [231, 735], [375, 512], [290, 722], [234, 612], [253, 577], [375, 535], [375, 535], [234, 612], [330, 523], [376, 466], [376, 466], [265, 517], [232, 668], [238, 742], [283, 719]

In [97]:
a = np.array(sorted(all_point))
s = []
for i in range(len(a)-1):
    m = cv2.absdiff(a[i+1],a[i])
    if(m[0]>2 or m[1]>2):
        s.append(a[i])
s[0]

array([  9, 534])

In [110]:
# For showing difeerent color on extreme point
origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp')
edge1 = origin_img[2744:3136,1824:2736]

for i in range(len(s)):
    cv2.circle(edge1,(s[i][1],s[i][0]),10,[0,255,0],2)
    edge1[s[i][0],s[i][1]] = [0,0,255]


# s = s + np.array([2744,1824])
cv2.imwrite('one_point_color.png',edge1)

True

In [ ]:
# For showing difeerent color on extreme point
origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
edge1 = origin_img[2744:3136,1824:2736]

for i in range(len(s)):
    edge1[s[i][0],s[i][1]] = 255


s = s + np.array([2744,1824])
# cv2.imwrite('one_point.png',edge1)

In [ ]:
Anna2_img = "/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp"
Anna2_img = cv2.imread(Anna2_img,0)

Anna3_img = "/home/office5/Image_Code/Ketan_sir/data/Anna3.bmp"
Anna3_img = cv2.imread(Anna3_img,0)


second_img_point = []
for i in range(len(s)):
    ori_img_loc_x = s[i][1]
    ori_img_loc_y = s[i][0]
    x1_f = ori_img_loc_x - 5
    x2_f = ori_img_loc_x + 5
    y1_f = ori_img_loc_y - 5
    y2_f = ori_img_loc_y + 5

    img_crop1  = Anna2_img[y1_f:y2_f,x1_f:x2_f]
    img_first = img_crop1.astype(np.int8)
    print("Original",ori_img_loc_y,ori_img_loc_x)

    x1_s = ori_img_loc_x - 10
    x2_s = ori_img_loc_x + 10
    y1_s = ori_img_loc_y - 10
    y2_s = ori_img_loc_y + 10

    img_crop2  = Anna3_img[y1_s:y2_s,x1_s:x2_s]
    img_snd = img_crop2.astype(np.int8)

    dict1 = {}
    for i in range(10):
        for j in range(10):
            dict1[i,j] = np.mean(np.abs(img_first  - img_snd[i:i+10,j:j+10]))
    # print(dict1)
    i,j = min(dict1, key=dict1.get)

    print(j,i)
    s_img_extr_point_x = x1_s + 10 - 5 + j # 10 for each block of first image and -5 for adding from width and height of first image
    s_img_extr_point_y =y1_s + 10 - 5 + i

    second_img_point.append([s_img_extr_point_y,s_img_extr_point_x])
    print("From second image",s_img_extr_point_y,s_img_extr_point_x)

In [20]:
import numpy as np
import cv2 as cv

image = "/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp"
img = cv.imread(image)
img = img[2744:3136,1824:2736]
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,5,0.04)
#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]
# cv.imshow('dst',img)
# cv.waitKey(0)
# cv.destroyAllWindows()

[[ 2.6466370e-01  2.6466370e-01  2.1985457e+00 ...  1.0043945e+00
   4.5692384e-01  9.7620249e-02]
 [ 2.6466370e-01  2.6466370e-01  2.1985457e+00 ...  1.0043945e+00
   4.5692384e-01  9.7620249e-02]
 [ 2.5457823e-01  2.5457823e-01  4.3144941e+00 ...  1.4542884e+00
   1.0066385e+00  1.7680769e+00]
 ...
 [ 6.1816400e+05  6.1816400e+05 -1.0384356e+06 ... -1.1938042e+06
  -4.3097325e+06 -3.5579662e+05]
 [ 6.6613556e+05  6.6613556e+05 -5.7437819e+05 ... -3.4787970e+06
  -9.3589180e+06 -5.6774300e+05]
 [ 1.2822947e+05  1.2822947e+05 -2.6812844e+05 ... -5.0411265e+06
  -1.1177601e+07 -1.4180332e+06]]


In [103]:
import numpy as np
import cv2 as cv
image = "/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp"
img_ = cv.imread(image)
gray_ori = cv.cvtColor(img_,cv.COLOR_BGR2GRAY)
img = img_[2744:3136,1824:2736]
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# find Harris corners
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)
dst = cv.dilate(dst,None)
ret, dst = cv.threshold(dst,200,255,0)
dst = np.uint8(dst)
# find centroids
ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)
# define the criteria to stop and refine the corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]
cv.imwrite('subpixel5.png',img)

True

In [104]:
points = np.array(np.int0(corners))

In [105]:
len(points)

451

In [106]:
import cv2

In [69]:
cv2.absdiff(np.array(gray_ori[points[i][0]-1][points[i][1]]) , np.array(gray_ori[points[i][0]+1][points[i][1]]))

array([[2]], dtype=uint8)

In [86]:
points[:,[0,1]] = points[:,[1,0]]

In [108]:
count=0
for i in range(len(points)):
    # try:
    upp_down_a = cv2.absdiff(np.array(gray_ori[points[i][0]-1][points[i][1]]) , np.array(gray_ori[points[i][0]+1][points[i][1]]))
    left_right_b = cv2.absdiff(np.array(gray_ori[points[i][0]][points[i][1]-1]),np.array(gray_ori[points[i][0]][points[i][1]+1]))
    
    # up_right = cv2.absdiff(np.array(gray_ori[points[i][0]-1][points[i][1]]) , np.array(gray_ori[points[i][0]][points[i][1]+1]))
    # left_down = cv2.absdiff(np.array(gray_ori[points[i][0]][points[i][1]-1]),np.array(gray_ori[points[i][0]+1][points[i][1]]))
    
    # print(upp_down_a,left_right_b)
    if(upp_down_a>=4 and left_right_b>=2):
        gray[points[i][1],points[i][0]] = 255
    #     # origin_img[points[i][0],points[i][1]] = 255

        # all_point.append([points[i][0],points[i][1]])
        count+=1
    # except:
    #     pass

print(count)

4


In [109]:
cv2.imwrite('combine.png',gray)

True